# Geomechanical Injection Scenario Toolkit (GIST)

#Disclaimer
GIST aims to give the _gist_ of a wide range of potential scenarios and aid collective decision making when responding to seismicity.

The results of GIST are entirely dependent upon the inputs provided, which may be incomplete or inaccurate.

There are other potentially plausible inducement scenarios that are not considered, including fluid migration into the basement, 
out-of-zone poroelastic stressing, or hydraulic fracturing.

None of the individual models produced by GIST accurately represent what happens in the subsurface and cannot be credibly used 
to accurately assign liability or responsibility for seismicity.

"All models are wrong, but some are useful" - George Box, 1976

## Prerequisites

Assumes InjectionSQLScheduled completed successfully and injection data are sampled uniformly in time

##Install Dependencies
- geopandas
- gistMC.py
- eqSQL.py
- gistPlots.py
- numpy
- scipy
- pandas


In [0]:
%restart_python

In [0]:
%run "/Workspace/_utils/Utility_Functions"

In [0]:
!pip install geopandas
!pip install geodatasets
!pip install contextily
#! pip install folium matplotlib mapclassify contextily

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 120.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.9/23.9 MB 79.3 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 3.8 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

##Paths

In [0]:
# Paths
homePath='/Workspace/Users/bill.curry@exxonmobil.com'
# Injection data path 
injPath=homePath+'/injection/WeeklyRun/ScheduledOutput'
# GIST library path
gistPath=homePath+'/GIST'

##Libraries

- numpy
- scipy
- pandas
- matplotlib
- geopandas
- pyspark


In [0]:
import sys
sys.path.append(gistPath+'/lib')

In [0]:
#Databricks-specific
#import dataBricksConfig as db
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("eqSQL").getOrCreate()

In [0]:
import numpy as np
import pandas as pd
import os
import gistMC as gi
import eqSQL as es
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
#import contextily as cx

In [0]:
import gistPlots as gp

#1. Select Event

In [0]:
eventID='texnet2024oqfb'

In [0]:
runPath=gistPath+'/runs/'+eventID+'/'
os.makedirs(runPath, exist_ok=True)

In [0]:
eqs=es.eqSQL()
EQDF=eqs.getEarthquake(eventID)
EQDF=EQDF.rename(columns={'LatitudeErrorKm':'LatitudeError','LongitudeErrorKm':'LongitudeError','EventId':'EventID'})
# What about the other fault plane solution? - 251 / 36 / -97
EQDF['Strike']=80.
EQDF['Dip']=55.
EQDF['Rake']=-85.
# We could make use of the Rake
EQDF.info()
eq=EQDF.loc[0]

getEarthquake:     SeismicEventId  ... B3RecordDeletedUTCDateTime
0         2481836  ...                        NaT

[1 rows x 32 columns]
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 38 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   SeismicEventId              1 non-null      int64         
 1   DataSource                  1 non-null      object        
 2   DataSourceUrl               0 non-null      object        
 3   EventID                     1 non-null      object        
 4   EventTimeUtc                1 non-null      datetime64[ns]
 5   EventTimeInLocalTimeZone    1 non-null      object        
 6   EventTimeZone               1 non-null      object        
 7   EventType                   1 non-null      object        
 8   DepthKm                     1 non-null      float64       
 9   DepthErrorKm                1 non-null     

In [0]:
EQDF.to_csv(runPath+'/EQ.csv')

#2. Initial Run

##2.1 Subsurface Model Parameters

###Inputs:

In [0]:
# Binary deep/shallow parameter
deepOrShallow='deep'
# Depth from surface (ft)
# Distinguishes deep/shallow where we don't have a horizon 
depthCutoff=8000.

In [0]:
nRealizations=500

In [0]:
# Water density minimum/maximum (kg/m3)
WaterDensityMin=1015.
WaterDensityMax=1025.
# Water viscosity minimum/maximum (Pa.s)
WaterViscosityMin=0.000799
WaterViscosityMax=0.000801
# Fluid Compressibility minimum/maximum (1/Pa)
FluidCompressibilityMin=0.000000000359
FluidCompressibilityMax=0.000000000361

In [0]:
# Porosity in percent
PorosityPercentMin=3.
PorosityPercentMax=15.

# Permeability in millidarcies
PermMDMin=5.
PermMDMax=500.

# Thickness in feet
ThicknessFTMin=200.
ThicknessFTMax=2000.

# Vertical compressibility minimum/maximum (1/Pa)
VerticalCompressibilityMin=0.00000000107
VerticalCompressibilityMax=0.00000000109

In [0]:
# Verbosity - 0=silent, 1=some, 2=lots
verb=0
# Minimum Pressure change to care about in PSI
dPCutoff=0.5
# Maximum Number of wells to plot
nWells=50
# How far back in time to plot in years
minYear=-40

### Set Interval-Specific Paths...


In [0]:
# Set an output directory for this earthquake and this interval
runIntervalPath=runPath+deepOrShallow+'/'
# Make directory if it doesn't exist
os.makedirs(runIntervalPath, exist_ok=True)
################################################
# Output prefix for realizations of parameters #
################################################
RealizationPrefix=runIntervalPath+'MC'

In [0]:
# Point to appropriate well and injection files
if deepOrShallow=='deep':
  WellFile=injPath+'/deep.csv'
  InjFile=injPath+'/deepReg.csv'
elif deepOrShallow=='shallow':
  WellFile=injPath+'/shallow.csv'
  InjFile=injPath+'/shallowReg.csv'

In [0]:
# Oversampling of time axis (poroelastic-only)
nTimeBins=21
# Poroelastic parameters - for in-zone poroelasticity in v2
ShearModulusMin=4e9
ShearModulusMax=6e9
PoissonsRatioDrainedMin=0.295
PoissonsRatioDrainedMax=0.305
PoissonsRatioUndrainedMin=0.305
PoissonsRatioUndrainedMax=0.315
BiotsCoefficientMin=0.26
BiotsCoefficientMax=0.36
# Fault parameters (poroelastic-only)
FaultFrictionCoeffMin=0.55
FaultFrictionCoeffMax=0.65
RockFrictionCoeffMin=0.55
RockFrictionCoeffMax=0.65

##2.2 Results

###Compute

In [0]:
gist=gi.gistMC(nReal=nRealizations,
                ntBin=nTimeBins)
gist.initPP(rho0_min=WaterDensityMin,
             rho0_max=WaterDensityMax,
             phi_min=PorosityPercentMin,
             phi_max=PorosityPercentMax,
             kMD_min=PermMDMin,
             kMD_max=PermMDMax,
             h_min=ThicknessFTMin,
             h_max=ThicknessFTMax,
             alphav_min=VerticalCompressibilityMin,
             alphav_max=VerticalCompressibilityMax,
             beta_min=FluidCompressibilityMin,
             beta_max=FluidCompressibilityMax)

In [0]:
gist.addWells(WellFile,InjFile,verbose=verb)
if verb>0:
  gist.wellDF.info()
  EQDF.info()

In [0]:
selectedWellsDF,ignoredWellsDF,injDF=gist.findWells(eq,PE=False,verbose=verb)

In [0]:
scenarioDF=gist.runPressureScenarios(eq,selectedWellsDF,injDF,verbose=verb)

In [0]:
filteredDF,orderedWellList=gi.summarizePPResults(scenarioDF,selectedWellsDF,threshold=dPCutoff,nOrder=nWells,verbose=verb)

In [0]:
diffRange=(min(gist.diffPPVec),max(gist.diffPPVec))
rtDF,mergedWellsDF = gi.prepRTPlot(selectedWellsDF,ignoredWellsDF,minYear,diffRange,clipYear=False)

In [0]:
winWellsDF,winInjDF=gi.getWinWells(filteredDF,selectedWellsDF,injDF)

In [0]:
# Slower implementation:
#scenarioTSDF,dPTimeSeries,wellIDs,dayVec = gist.runPressureScenariosTimeSeries(eq,winWellsDF,winInjDF,verbose=verb)
scenarioTSDF,dPTimeSeries,wellIDs,dayVec = gist.runPressureScenariosTimeSeriesTest(eq,winWellsDF,winInjDF,verbose=verb)

runPressureScenariosTimeSeriesTest Well  1  of  39
runPressureScenariosTimeSeriesTest Well  11  of  39
runPressureScenariosTimeSeriesTest Well  21  of  39
runPressureScenariosTimeSeriesTest Well  31  of  39


In [0]:
dPTimeSeriesSum=dPTimeSeries.sum(axis=1)

###Output

In [0]:
gist.writeRealizations(runIntervalPath+'PorePressureRealizations.csv')

In [0]:
selectedWellsDF.to_csv(runIntervalPath+'selectedWells.csv')
ignoredWellsDF.to_csv(runIntervalPath+'ignoredWells.csv')
injDF.to_csv(runIntervalPath+'inj.csv')

In [0]:
scenarioDF.to_csv(runIntervalPath+'scenarios.csv')

In [0]:
filteredDF.to_csv(runIntervalPath+'filteredScenarios.csv')
pd.Series(data=orderedWellList).to_csv(runIntervalPath+'wellOrder.csv')

In [0]:
mergedWellsDF.to_csv(runIntervalPath+'RTwells.csv')
rtDF.to_csv(runIntervalPath+'RTDF.csv')

In [0]:
scenarioTSDF.to_csv(runIntervalPath+'materialScenarios.csv')
np.savez_compressed(runIntervalPath+'timeSeries.npz', deltaPP=dPTimeSeries,dayVec=dayVec,wellIDs=wellIDs)

In [0]:
np.savez_compressed(runIntervalPath+'timeSeriesSum.npz', deltaPP=dPTimeSeriesSum,dayVec=dayVec,wellIDs=wellIDs)

#3. Correct Data

##3.1 Export Disposal Data

##3.2 Import Updated Disposal Data

In [0]:
# Input a single file with a lot of columns
# newWellInfo=pd.read_csv('path_to_new_well_file')
# Get wells with a wellID.unique() call
# Separate out dates, wellIDs, and rates to a separate injection file

#4. Updated Analysis

##Repeat Steps 2.1 and 2.2
Updated disposal data

Refined parameters

#5. Forecast

##5.1 Input Forecast Times

In [0]:
# End Year of time series forecast
EndYear=2030
# Time to calculate rate distributions


##5.2 Distributions of rates for each well

In [0]:
# Rerun select wells with an updated earthquake time set to the last day of EndYear
future=eq.copy()
future['Origin Date']=pd.to_datetime('12-31-'+str(EndYear))
# newSelectedWellsDF,ignoredWellsDF,injDF=gist.findWells(future,PE=False,verbose=verb)
# Merge udpated well dataframe with prior unselected wells
# Re-generate an R-T plot here. Should it have two sets of curves?

SeismicEventId                                      2481836
DataSource                        TexNet Earthquake Catalog
DataSourceUrl                                          None
EventID                                      texnet2024oqfb
EventTimeUtc                            2024-07-27 02:11:07
EventTimeInLocalTimeZone               2024-07-26T21:11:07Z
EventTimeZone                                           CST
EventType                                        Earthquake
DepthKm                                            8.837891
DepthErrorKm                                       0.636678
Magnitude                                           1.64016
MagnitudeError                                          NaN
MagnitudeType                                    ml(texnet)
Location                                      Western Texas
Status                                                final
Latitude                                          32.144165
LatitudeError                           

In [0]:
# I need new code here that calculates the time derivatives at a future point

In [0]:
# Here is where the input well rates come into play

In [0]:
# And here is the future time forecast